In [3]:
#Load libraries
import pandas as pd
import sys
from sqlalchemy import create_engine, text
from configobj import ConfigObj
import contextlib
from functools import reduce
import os

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from functions.utils import database

In [4]:
## define paths and variables

config_path = "/home/jovyan/shared/rural_planner_refactoring/config_files_test/config_co"

parser = ConfigObj(config_path)

sql_path =  parser['sql_path']
country_folder = parser['country_folder']
schema = parser['segmentation_params']['schema']


output_table = parser['segmentation_params']['output_table']
table_settlements = parser['segmentation_params']['table_settlements']
table_coverage = parser['segmentation_params']['table_coverage']

In [6]:
#Import the data frames to be used

query_path = sql_path + '/' + country_folder + '/' + 'segmentation_get_unserved.sql'
with open(query_path) as file, database(parser) as db:
    query = file.read()
    query_formatted = query.format(schema = schema, table_coverage = table_coverage, table_settlements = table_settlements)
    df_tef_organic = pd.read_sql_query(query_formatted, db)
        

query_path = sql_path + '/' + country_folder + '/' + 'segmentation_get_overlay.sql'
with open(query_path) as file, database(parser) as db:
    query = file.read()
    query_formatted = query.format(schema = schema, table_coverage = table_coverage, table_settlements = table_settlements)
    df_overlay_2g = pd.read_sql_query(query_formatted, db)
        

query_path = sql_path + '/' + country_folder + '/' + 'segmentation_get_greenfield.sql'
with open(query_path) as file, database(parser) as db:
    query = file.read()
    query_formatted = query.format(schema = schema, table_coverage = table_coverage, table_settlements = table_settlements)
    df_greenfield = pd.read_sql_query(query_formatted, db)
    

query_path = sql_path + '/' + country_folder + '/' + 'segmentation_get_fully_unconnected.sql'
with open(query_path) as file, database(parser) as db:
    query = file.read()
    query_formatted = query.format(schema = schema, table_coverage = table_coverage, table_settlements = table_settlements)
    df_fully_unconnected = pd.read_sql_query(query_formatted, db)
    
#Merge data frames
dataframes = [df_tef_organic, df_overlay_2g, df_greenfield, df_fully_unconnected]
df_final = reduce(lambda left,right: pd.merge(left,right, on='settlement_id', how = 'left'), dataframes)

#Upload output to database
with database(parser) as db:    
    df_final.to_sql(output_table, con=db, if_exists = 'replace', schema = schema, index = False)

        